<a href="https://colab.research.google.com/github/abhijain864/VTON-Intern/blob/main/Hugoboss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import requests
from requests.exceptions import ConnectionError
import os
from bs4 import BeautifulSoup
import re
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
}
import numpy as np

men_polo-shirts   300
men_t-shirts   352
men_shirts   330
men_knitwear   183
men_jeans   183
men_trousers   313
men_tailored-jackets   98
men_suits   78
men_mix-match   114
men_tracksuits   252
men_jackets-coats   111
men_coats   32
women_dresses   135
women_blazers   55
women_t-shirts   111
women_tops   57
women_knitwear   132
women_sweatshirts   57
women_tracksuits   23
women_trouser-suits-skirt-suits   43
women_trousers   123
women_jeans   39
women_skirts   71
women_leather-jackets   16
women_jackets-coats   17
women_coats   19


In [ ]:
# men_category = ['polo-shirts','t-shirts','shirts','knitwear','jeans','trousers','tailored-jackets','suits','mix-match','tracksuits','jackets-coats','coats']
# women_category = ['dresses','blazers','t-shirts','tops','knitwear','sweatshirts','tracksuits', 'trouser-suits-skirt-suits','trousers','jeans','skirts',
# 'leather-jackets','jackets-coats','coats']
# def no_of_prod_dict(gender,list_of_category):
#   for ctg in list_of_category:
#     url = 'https://www.hugoboss.com/in/en/' + gender + '-' +ctg+'/'
#     r = requests.get(url, headers=HEADERS) ; 
#     soup = BeautifulSoup(r.text,'html.parser') ; 
#     temp = soup.find('span',class_='search-result-options__brand-badge').text
#     no_of_products = int(re.split(r'\n +',temp)[1])
#     print(gender+'_'+ctg,' ',no_of_products)
# no_of_prod_dict('men',men_category)
# no_of_prod_dict('women',women_category)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [ ]:
%cd /content/drive/My Drive/Web_scraping/Hugoboss/Hugoboss_images
# %cd /content/drive/My Drive/Trial_purpose

/content/drive/My Drive/Web_scraping/Hugoboss/Hugoboss_images


In [ ]:
def delete_imgs(sure=False):
  if sure:
    for pic in os.listdir('/content/drive/My Drive/Web_scraping/Hugoboss/Hugoboss_images'):
      os.remove(pic)

In [ ]:
def func_img_category(url):
  ctg = re.split('/',url)[5]
  gender = re.split(r'-',ctg)[0]
  modified = re.sub(gender+'-',gender+'_', ctg + '_')
  return modified

In [ ]:
def func_description(img_page_link):
  return(re.split('/',img_page_link)[5])

In [ ]:
def save(img_category,img_urls,description,want_descrip = False):
  global count
  if len(img_urls)<4 :
    return
  file_name = img_category + str(count) + '_0'
  # with open(file_name + '.jpg','wb') as f:
  #   im = requests.get(img_urls[-1],headers=HEADERS)
  #   f.write(im.content)   
  file_name = img_category + str(count) + '_1'
  # with open(file_name + '.jpg','wb') as f:
  #   im = requests.get(img_urls[1],headers=HEADERS)
  #   f.write(im.content) 
  file_name = img_category + str(count) + '_2'
  # with open(file_name + '.jpg','wb') as f:
  #   im = requests.get(img_urls[0],headers=HEADERS)
  #   f.write(im.content) 
  if want_descrip:
    print(img_category + str(count),end=' ')
    print(' name is ',description)
  else:
    print(img_category + str(count))
  count=count+1

In [ ]:
def modify_url(urls):
  changed_to = []
  for we_have in urls:
    we_have = re.sub(r're_fullPageZoom','large',we_have)
    we_have = re.sub(r'&amp;','&',we_have)
    we_have = re.sub(r'\$large\$&qlt=85&fit','$large$=&fit',we_have)
    we_have = re.sub(r'&wid=70&hei=106','&wid=768&qlt=80',we_have)
    changed_to.append(we_have)
    # print(we_have)
  return changed_to

In [ ]:
def generate_image_urls(url):
  try:
    r = requests.get(url, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(60) 
    r = requests.get(url, headers=HEADERS)

  soup = BeautifulSoup(r.text,'html.parser')
  urls = []
  for tag in soup.find_all('li',class_='stage__images-thumbnail-item js-nav-item '):
    url = tag.img['src']
    urls.append(url)
  modified_urls = modify_url(urls)
  return modified_urls

In [ ]:
def scroll_down(url):
  # print(url)
  scroll_down_url_set = []
  scroll_down_url_set.append(url)
  try:
    r = requests.get(url, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(60) 
    r = requests.get(url, headers=HEADERS)



  soup = BeautifulSoup(r.text,'html.parser') 
  temp = soup.find('span',class_='search-result-options__brand-badge').text
  no_of_products = int(re.split(r'\n +',temp)[1])
  for i in range(int(np.ceil(no_of_products/60))-1):
    new_url =  url + '?start=' + str(60*(i+1)) + '&sz=60'
    # print(new_url)
    scroll_down_url_set.append(new_url)
  return scroll_down_url_set

In [ ]:
def scrape(url,want_descrip=False):
  img_category = func_img_category(url) # Gives for ex Mens_jackets
  print('sraping_',img_category) 

  scroll_down_url_set = scroll_down(url) # Set of all urls upon scrolling through the whole webpage 


  for url in scroll_down_url_set:
    try:
      r = requests.get(url, headers=HEADERS) 
    except ConnectionError as err:
      time.sleep(60)
      r = requests.get(url, headers=HEADERS)


    soup = BeautifulSoup(r.text,'html.parser')
    for link in soup.find_all('a', class_="product-tile-default__title-link js-product-tile-link widget"):
      img_page_link = 'https://www.hugoboss.com'+link.get('data-url')
      # print(url)
      description = func_description(img_page_link) # In case you want product name for ex ''Wool & Mohair Blend Blazer''
      img_urls = generate_image_urls(img_page_link)
      save(img_category,img_urls,description,want_descrip)

In [ ]:
def url_generator(gender,list_of_category):
  urls = []
  for ctg in list_of_category:
    urls.append('https://www.hugoboss.com/in/en/' + gender  + '-' +ctg+'/')
  return urls

In [ ]:
def scrape_these_urls(urls):
  for url in urls:
    scrape(url,want_descrip=True)

In [ ]:
#women_category = ['dresses','blazers','t-shirts','tops','knitwear','sweatshirts','tracksuits', 'trouser-suits-skirt-suits','trousers','jeans','skirts',
# 'leather-jackets','jackets-coats','coats']

In [ ]:
# Run this only one time, add additional categories in another snippet
# count=1
# delete_imgs()
scrape_these_urls(url_generator('men',['jackets-coats','trousers','mix-match','suits','t-shirts','shirts','coats']))
scrape_these_urls(url_generator('women',['dresses','blazers','t-shirts','tops','knitwear','sweatshirts','tracksuits', 'trouser-suits-skirt-suits','trousers','jeans','skirts',
'leather-jackets','jackets-coats','coats']))

sraping_ men_jackets-coats_
men_jackets-coats_712  name is  biker-style-waxed-leather-jacket-with-quilting
men_jackets-coats_713  name is  water-repellent-zip-up-jacket-with-logo-details
men_jackets-coats_714  name is  slim-fit-bomber-jacket-in-suede-with-sleeve-pocket
men_jackets-coats_715  name is  lightweight-quilted-jacket-in-100-recycled-materials
men_jackets-coats_716  name is  slim-fit-leather-jacket-with-contrast-lining
men_jackets-coats_717  name is  packable-down-gilet-with-water-repellent-outer
men_jackets-coats_718  name is  regular-fit-denim-jacket-with-lasered-artwork
men_jackets-coats_719  name is  relaxed-fit-jacket-in-coated-cotton-with-exclusive-logo
men_jackets-coats_720  name is  hooded-water-repellent-jacket-with-logo-details
men_jackets-coats_721  name is  water-repellent-jacket-with-logo-statements
men_jackets-coats_722  name is  natural-denim-jacket-with-white-gold-effect-trims
men_jackets-coats_723  name is  regular-fit-water-repellent-jacket-with-statement-pri

In [ ]:
# Add additional categories you want to scrape here
#scrape_these_urls('gender','give more categories')



---



---



#The below code is just for debugging

In [ ]:
import requests
import os
from bs4 import BeautifulSoup
import re
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
}
import numpy as np

In [ ]:
url = 'https://www.hugoboss.com/in/en/men-t-shirts/?start=240&sz=60'
r = requests.get(url, headers=HEADERS) ; soup = BeautifulSoup(r.text,'html.parser')
for link in soup.find_all('a', class_="product-tile-default__title-link js-product-tile-link widget"):
  print('https://www.hugoboss.com'+link.get('data-url'))

https://www.hugoboss.com/in/en/regular-fit-logo-t-shirt-in-bionic%C2%AE-single-jersey/hbeu50447948_618.html
https://www.hugoboss.com/in/en/regular-fit-t-shirt-with-raw-cut-edges/hbeu50385535_100.html
https://www.hugoboss.com/in/en/colour-block-logo-t-shirt-in-stretch-cotton-jersey/hbeu50447935_031.html
https://www.hugoboss.com/in/en/slim-fit-t-shirt-in-stretch-pique-with-tonal-logo/hbeu50448694_001.html
https://www.hugoboss.com/in/en/slim-fit-t-shirt-with-long-sleeves-in-waffle-cotton/hbeu50401846_250.html
https://www.hugoboss.com/in/en/long-sleeved-relaxed-fit-t-shirt-with-coastal-prints/hbeu50450920_610.html
https://www.hugoboss.com/in/en/colour-block-logo-t-shirt-in-stretch-cotton-jersey/hbeu50447935_410.html
https://www.hugoboss.com/in/en/regular-fit-t-shirt-with-contrast-detail/hbeu50245195_032.html
https://www.hugoboss.com/in/en/two-pack-of-performance-t-shirts-in-s.cafe%C2%AE-jersey/hbeu50446125_960.html
https://www.hugoboss.com/in/en/regular-fit-cotton-jersey-t-shirt-with-multi



---



---



In [ ]:
url = 'https://www.hugoboss.com/in/en/packable-down-gilet-with-water-repellent-outer/hbeu50448705_489.html?cgid=21200'
r = requests.get(url, headers=HEADERS) ; soup = BeautifulSoup(r.text,'html.parser')
urls = []
for tag in soup.find_all('li',class_='stage__images-thumbnail-item js-nav-item '):
  url = tag.img['src']
  urls.append(url)
modified_urls = modify_url(urls)

https://images.hugoboss.com/is/image/boss/hbeu50448705_489_350?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80
https://images.hugoboss.com/is/image/boss/hbeu50448705_489_300?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80
https://images.hugoboss.com/is/image/boss/hbeu50448705_489_360?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80
https://images.hugoboss.com/is/image/boss/hbeu50448705_489_340?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80
https://images.hugoboss.com/is/image/boss/hbeu50448705_489_341?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80
https://images.hugoboss.com/is/image/boss/hbeu50448705_489_100?$large$=&fit=crop%2C1&align=1%2C1&extendN=0.05%2C0.05%2C0.05%2C0.05&wid=768&qlt=80


In [ ]:
modified_urls

['https://images.hugoboss.com/is/image/boss/hbeu50448705_489_350?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80',
 'https://images.hugoboss.com/is/image/boss/hbeu50448705_489_300?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80',
 'https://images.hugoboss.com/is/image/boss/hbeu50448705_489_360?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80',
 'https://images.hugoboss.com/is/image/boss/hbeu50448705_489_340?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80',
 'https://images.hugoboss.com/is/image/boss/hbeu50448705_489_341?$large$=&fit=crop%2C1&align=1%2C1&wid=768&qlt=80',
 'https://images.hugoboss.com/is/image/boss/hbeu50448705_489_100?$large$=&fit=crop%2C1&align=1%2C1&extendN=0.05%2C0.05%2C0.05%2C0.05&wid=768&qlt=80']

In [ ]:
def modify_url(urls):
  changed_to = []
  for we_have in urls:
    we_have = re.sub(r're_fullPageZoom','large',we_have)
    we_have = re.sub(r'&amp;','&',we_have)
    we_have = re.sub(r'\$large\$&qlt=85&fit','$large$=&fit',we_have)
    we_have = re.sub(r'&wid=70&hei=106','&wid=768&qlt=80',we_have)
    changed_to.append(we_have)
    print(we_have)
  return changed_to



---



---



In [ ]:
url = 'https://www.hugoboss.com/in/en/men-beachwear/'
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')
temp = soup.find('span',class_='search-result-options__brand-badge').text
num_of_products = int(re.split(r'\n +',temp)[1])
print(num_of_products,' type is ',type(num_of_products))

107  type is  <class 'int'>




---



---



In [ ]:
for page in soup.find_all('a',class_='button button--pagingbar font__nav-link '):
  print(page)



---



---

